In [ ]:
pip install langchain_community

In [ ]:
from langchain_community.llms import Ollama

In [ ]:
llm = Ollama(model="llama2")

In [ ]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

In [ ]:
pip install pandas

In [ ]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2023_2024.csv")
df.head()

In [ ]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

In [ ]:
unique_transactions[1:10]

In [ ]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

In [ ]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df 

In [ ]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

In [ ]:
categories_df_all

In [ ]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

In [ ]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

In [ ]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

In [ ]:
# Merge the categories_df_all with the transactions_2023_2024.csv dataframe (df)
df = pd.read_csv("transactions_2023_2024.csv")
# Reset the index of both DataFrames to ensure alignment by index
df.reset_index(drop=True, inplace=True)
categories_df_all.reset_index(drop=True, inplace=True)

# Concatenate DataFrames horizontally (axis=1)
df_combined = pd.concat([df, categories_df_all], axis=1)

# Save the combined DataFrame to a new CSV file if needed
df_combined.to_csv("combined_transactions_categories.csv", index=False)

# Display the first few rows of the combined DataFrame
print(df_combined.head())